<a href="https://colab.research.google.com/github/suhayb-h/Acute-Lymphoblastic-Leukemia-Classifier/blob/main/New_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This final notebook outlines building a transfer learning neural network to identify lymphoblasts as being cancerous or normal.

Transfer learning requires a pre-trained neural network to serve as a starting point in model construction. In the context of image classification, a ready-made model trained on a large and generalized dataset will serve as a generic model of the visual world. New layers can be added to the base model and only these new layers will be trained towards the specified task of interest. In the case of this project, the task is to correctly identify images in the ISBI-2019 dataset as cancerous or normal. As an analogy, the machine needs to be 'specialized' much like physicians, clinicians and scientists need to specialize their knowledge of the entire visual world towards interpreting images of cells and microscopic features.

A pre-trained image classifier model is a documented and openly available network that was previously trained on a extensive dataset of random images. For this project, a pre-trained model will be used directly from TensorFlow Hub, a repository of pre-trained TensorFlow models. Once a model has been loaded, new layers were added to allow for transfer learning to fine-tune the model to serve as an ALL classifier model.

There are two ways to customize the pre-trained model:

*   Feature Extraction: Use the representations learned by a previous network to extract meaningful features from new samples. You simply add a new classifier, which will be trained from scratch, on top of the pretrained model so that you can repurpose the feature maps learned previously for the dataset.

*   Fine-Tuning: Unfreeze a few of the top layers of a frozen model base and jointly train both the newly-added classifier layers and the last layers of the base model. This allows us to "fine-tune" the higher-order feature representations in the base model in order to make them more relevant for the specific task.

Data augmentation serves as a central principal that will be applied to all models in this project. The purpose of this practice was to increase the diversity of the training set by applying random, yet realistic, transformations to the dataset to create new images for the dataset. These transformations include image flipping and image rotation.

Compose the model

*   Load in the pretrained base model and corresponding pretrained weights
*   Stack the classification layers on top

In [1]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime

%load_ext tensorboard

TensorFlow hub has a variety of models available to be used for a variety of purposes. 154 models are available within this repository for image classification alone. When selecting for models trained on the largest possible generalized dataset (identified as the ImageNet ILSVRC dataset) and the most recent version of the TensorFlow library, 65 models were available. Of the myriad of model architectures available, the most complex and latest variant ResNet model was selected as the pre-trained model to be used for the ALL classifier. This model is the ResNet V2 152 model.

In [8]:
classifier_model = "https://tfhub.dev/google/imagenet/resnet_v2_152/classification/5"

classifier = \
tf.keras.Sequential(
    hub.KerasLayer(
        classifier_model, 
        input_shape = (224,224,3,)
        ))

In [ ]:
data_root = tf.keras.utils.get_file(
  'flower_photos',
  'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

train_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size = (224, 224),
  batch_size = 32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size = (224, 224),
  batch_size = 32
)

In [ ]:
class_names = np.array(train_ds.class_names)
print(class_names)

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

In [ ]:
#AUTOTUNE = tf.data.AUTOTUNE
#train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
#val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Train the model


Evaluate model

In [ ]:
!pip install git+https://github.com/qubvel/classification_models.git

In [4]:
from classification_models.tfkeras import Classifiers

In [5]:
SeResNeXT, preprocess_input = Classifiers.get('seresnext50')
model = SeResNeXT(include_top = False, input_shape=(224, 224, 3), weights='imagenet')

104693760/104679448 [==============================] - 6s 0us/step
